In [3]:
import pandas as pd
from typing import List, Dict

df = pd.read_csv("restaurants_definitivo.csv", index_col = 0)
df

,Name,Address,Lat,Long,Number of Ratings,Restaurant Rating,Type of Restaurant,Number of Bookmarks,Price Range,Phone Number,Website,Instagram,Facebook,Timetable,url
id,,,,,,,,,,,,,,,
1,Freedom Cakes Diner,"Calle Cádiz 7,Madrid,Spain,28004",40.415876,-3.702805,470.0,4.5,['International'],766,Moderado,+34-910741347,http://www.freedomcakes.es/,http://www.instagram.com/freedomcakes,http://facebook.com/Freedomcakesvegan,"{'Monday': '08:00 - 00:00', 'Tuesday': '08:00 ...",https://www.happycow.net/reviews/freedom-cakes...
2,Mad Mad Vegan - Chueca,"Calle Pelayo 19,Madrid,Spain,28004",40.422698,-3.698609,410.0,5.0,"['American', 'Mexican']",612,Moderado,+34-914442077,https://madmadvegan.com/,http://www.instagram.com/madmadvegan,http://facebook.com/MADMADVEGAN,"{'Monday': '13:00 - 00:00', 'Tuesday': '13:00 ...",https://www.happycow.net/reviews/mad-mad-madri...
3,Frutas Prohibidas,"Calle del Conde Duque, 26,Madrid,Spain,28015",40.427467,-3.710349,241.0,5.0,"['International', 'Fusion']",555,Moderado,+34-911525533,http://www.frutasprohibidas.com/,http://www.instagram.com/frutasprohibidasbar,http://facebook.com/frutasprohibidasbar,"{'Monday': 'Closed', 'Tuesday': 'Closed', 'Wed...",https://www.happycow.net/reviews/frutas-prohib...
4,Mad Mad Vegan - Lavapiés,"Lavapiés 16,Madrid,Spain,28012",40.411113,-3.703039,216.0,4.5,"['American', 'Mexican']",465,Moderado,+34-910665299,https://madmadvegan.com/,http://www.instagram.com/madmadvegan,http://facebook.com/MADMADVEGAN,"{'Monday': '13:00 - 00:00', 'Tuesday': '13:00 ...",https://www.happycow.net/reviews/mad-mad-vegan...
5,Flower Burger,"Calle Luchana 13, Chamberí,Madrid,Spain,28010",40.430001,-3.701037,14.0,4.5,"['American', 'Western']",47,Barato,+34-910328968,https://www.flowerburger.it/,http://www.instagram.com/flowerburger_es,http://facebook.com/flowerburger.it,"{'Monday': '13:00 - 16:30, 19:30 - 23:30', 'Tu...",https://www.happycow.net/reviews/flower-burger...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,Morgano Crafted Beers,"Alamo 2, Boadilla del Monte, Madrid, Spain, 28660",40.405759,-3.876917,1.0,4.0,"['Middle Eastern', 'Spanish']",2,Barato,+34-695635423,No website,No instagram,http://facebook.com/MorganoCraftedBeers,"{'Monday': '19:00 - 01:00', 'Tuesday': '19:00 ...",https://www.happycow.net/reviews/morgano-craft...
745,Adagio Gelateria,"Avenida de Adolfo Suárez, 7, Boadilla Del Mont...",40.405413,-3.876999,1.0,4.0,['Italian'],0,No price range,+34-912859853,No website,http://www.instagram.com/adagiogelateria,http://facebook.com/adagiogelateria,"{'Monday': '10:00 - 22:30', 'Tuesday': '10:00 ...",https://www.happycow.net/reviews/adagio-gelate...
746,La Huella Vegana,"Calle de las Trece Rosas, 2,Rivas-vaciamadrid,...",40.376390,-3.540020,31.0,4.5,"['Western', 'Spanish']",42,Moderado,+34-917139958,http://www.lahuellavegana.com/,No instagram,http://facebook.com/lahuellavegana,"{'Monday': '08:30 - 23:00', 'Tuesday': '08:30 ...",https://www.happycow.net/reviews/la-huella-riv...


In [45]:
tipos_restaurantes: List[str] = ["Korean", "African", "American", "Asian", "Australian", "Brazilian", "British", "Caribbean", "Chinese",
                 "European", "French", "Fusion", "German", "Indian", "International", "Italian", "Japanese", "Latin",
                 "Mediterranean", "Mexican", "Middle Eastern", "Spanish", "Taiwanese", "Thai", "Vietnamese", "Western"]

total_tipos: Dict[str, Dict[str, float]]= {}

for tipo in tipos_restaurantes:
    filtro_tipo = df[df["Type of Restaurant"].str.contains(tipo, case=False, na=False)]
    numero_tipo = len(filtro_tipo)
    numero_reviews = int(filtro_tipo["Number of Ratings"].sum())

    distribucion = (filtro_tipo["Number of Ratings"]/numero_reviews)*filtro_tipo["Restaurant Rating"]

    if numero_reviews > 0:
        media_ponderada = round(float((distribucion).sum()), 3)
    else: 
        media_ponderada = 0

    total_tipos[tipo] = {
                         "total" : numero_tipo,
                         "reviews" : numero_reviews,
                         "MP rating" : media_ponderada
                         }


In [ ]:
total_reviews_ordenados = dict(sorted(total_tipos.items(), key= lambda item: item[1]["reviews"], reverse= True))
df_reviews_ordenados = pd.DataFrame.from_dict(total_reviews_ordenados, orient='index')
df_reviews_ordenados[:10]

,total,reviews,MP rating
Spanish,227,4898,4.419
International,101,2370,4.403
Mediterranean,120,2351,4.397
Western,80,1558,4.437
Fusion,61,1519,4.431
American,85,1356,4.490
European,124,1241,4.164
Italian,86,1078,4.565
Mexican,37,1006,4.572
Asian,86,412,3.823


In [48]:
df_reviews_ordenados[-10:]

,total,reviews,MP rating
African,7,25,3.920
Indian,7,22,3.818
French,6,19,3.211
Australian,2,18,3.806
Taiwanese,1,13,3.500
Korean,4,10,3.400
Caribbean,3,5,3.200
Vietnamese,3,5,3.600
German,2,1,4.000
British,1,0,0.000


In [ ]:
total_tipos_ordenados = dict(sorted(total_tipos.items(), key= lambda item: item[1]["total"], reverse= True))
df_tipos_ordenado = pd.DataFrame.from_dict(total_tipos_ordenados, orient='index')
df_tipos_ordenado[:10]


,total,reviews,MP rating
Spanish,227,4898,4.419
European,124,1241,4.164
Mediterranean,120,2351,4.397
International,101,2370,4.403
Asian,86,412,3.823
Italian,86,1078,4.565
American,85,1356,4.490
Western,80,1558,4.437
Fusion,61,1519,4.431
Japanese,54,271,3.873


In [50]:
df_tipos_ordenado[-10:]


,total,reviews,MP rating
Indian,7,22,3.818
French,6,19,3.211
Thai,5,26,3.500
Korean,4,10,3.400
Caribbean,3,5,3.200
Vietnamese,3,5,3.600
Australian,2,18,3.806
German,2,1,4.000
British,1,0,0.000
Taiwanese,1,13,3.500


In [52]:
total_rating_ordenados = dict(sorted(total_tipos.items(), key= lambda item: item[1]["MP rating"], reverse= True))
df_rating_ordenados = pd.DataFrame.from_dict(total_rating_ordenados, orient='index')
df_rating_ordenados[:10]

,total,reviews,MP rating
Middle Eastern,34,335,4.578
Mexican,37,1006,4.572
Brazilian,8,35,4.571
Italian,86,1078,4.565
American,85,1356,4.490
Western,80,1558,4.437
Fusion,61,1519,4.431
Spanish,227,4898,4.419
International,101,2370,4.403
Mediterranean,120,2351,4.397


In [53]:
df_rating_ordenados[-10:]

,total,reviews,MP rating
Indian,7,22,3.818
Australian,2,18,3.806
Chinese,19,246,3.803
Vietnamese,3,5,3.600
Taiwanese,1,13,3.500
Thai,5,26,3.500
Korean,4,10,3.400
French,6,19,3.211
Caribbean,3,5,3.200
British,1,0,0.000
